In [1]:
def ler_arquivo_para_lista(nome_arquivo):
    minha_lista = []
    with open(nome_arquivo, 'r') as file:
        for line in file:
            minha_lista.append(line.strip())
    return minha_lista

In [2]:
lista = ler_arquivo_para_lista("anvisa/01_texto.txt")
lista

['I - IDENTIFICAÇÃO DO MEDICAMENTO Neosaldina® comprimido revestido dipirona +mucato de isometepteno + cafeína Neosaldina® solução dipirona + cloridrato de isometepteno + cafeína APRESENTAÇÕES Comprimido revestido. Embalagens contendo 20, 30, 100, 200 ou 240 comprimidos revestidos. Solução. Embalagem contendo 01 frasco gotejador com 15mL . USO ORAL USO ADULTO E PEDIÁTRICO ACIMA DE 12 ANOS - Neosaldina®  comprimido revestido USO ADULTO E PEDIÁTRICO ACIMA DE 1 ANO - Neosaldina® solução COMPOSIÇÕES Cada comprimido revestido contém: dipirona------------------------------------------------------------------------------------------------------------300mg mucato de isometepteno------------------------------------------------------------------------------------------30mg cafeína anidra------------------------------------------------------------------------------------------------------30mg excipientes q.s.p. ----------------------------------------------------------------------------- comprimi

In [3]:
import bullama.util
from bullama.util import create_model_tokenizer
model_name = '7b'
output_dir = 'OUTPUTS'
model_dir = 'models'
exp_name = 'bulario_plaintext'

model, tokenizer = create_model_tokenizer(model_name, True, False)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/navarro/miniconda3/envs/py39/lib/python3.9/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [17]:
exp_name = 'bulario_plaintext'
trainer_sft.save_model(f"{model_dir}/{exp_name}")
tokenizer.save_pretrained(f"{model_dir}/{exp_name}")

('models/bulario_plaintext/tokenizer_config.json',
 'models/bulario_plaintext/special_tokens_map.json',
 'models/bulario_plaintext/tokenizer.json')

In [4]:
import pandas as pd
bulario_df = pd.read_csv('datasets/bulario_plain_text.csv', delimiter=';')
bulario_df = bulario_df.replace('\n','', regex=True)
bulario_df = bulario_df.replace('\u200b','', regex=True)


In [5]:
bulario_df.iloc[0].values

array([0, 'Abilify comprimidos',
       '   Abilify comprimidos de 5, 10, 15, 20 ou 30 mg é indicado no tratamento de esquizofrenia em adultos e adolescentes com mais de 15 anos.  Além disso, estes comprimidos de ingestão oral, também podem ser utilizados no tratamento e prevenção de episódios maníacos na perturbação bipolar 1 em adultos e adolescentes com mais de 13 anos.  Abilify comprimidos também pode ser encontrado na forma de comprimidos orodispersíveis que dissolvem na boca sem uso de água, sendo indicados para pacientes idosos, agitados ou com problemas para engolir.   As doses de Abilify variam de acordo com a idade do paciente e a doença a tratar, sendo que as orientações gerais incluem:  Adultos  Esquizofrenia: a dose inicial é de 10 a 15 mg uma vez por dia, independente das refeições. Já a dose de manutenção, normalmente, é de 15 mg por dia, podendo ser aumentada até 30 mg de acordo com indicação do médico.\tEpisódios maníacos na perturbação bipolar 1: a dose inicial é de 1

In [6]:
from datasets import Dataset

#dataset = Dataset.from_dict({'text': [''.join(lista)]})
dataset = Dataset.from_pandas(bulario_df)


In [7]:
dataset

Dataset({
    features: ['Unnamed: 0', 'Nome', 'texto'],
    num_rows: 1590
})

In [8]:
model = bullama.util.prepare_model(model)


In [11]:
from typing import Callable, Mapping
def tokenize(element: Mapping, tokenizer: Callable, context_length: int) -> str:
    inputs = tokenizer(element['texto'], truncation=True, return_overflowing_tokens=True, 
                       return_length=True, max_length=context_length)
    inputs_batch = []
    for length, input_ids in zip(inputs['length'], inputs['input_ids']):
        if length == context_length: # We drop the last input_ids that are shorter than max_length
            inputs_batch.append(input_ids)
    return {"input_ids": inputs_batch}

In [12]:
tokenized_dataset = dataset.map(tokenize, batched=True, fn_kwargs={'tokenizer': tokenizer, 'context_length': 2048},
                                        remove_columns=dataset.column_names)

Map:   0%|          | 0/1590 [00:00<?, ? examples/s]

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
training_args = TrainingArguments(

    per_device_train_batch_size = 1, #The batch size per GPU (default: 8)
    gradient_accumulation_steps = 1,
    warmup_steps = 0, # Number of steps used for a linear warmup from 0 to learning_rate (default: 0)
    num_train_epochs=3,
   # weight_decay=0.1,
    learning_rate = 3e-4,
    #fp16 = True, # Whether to use fp16 16-bit (mixed) precision training instead of 32-bit training.
    logging_steps = 1, # Number of update steps between two logs if logging_strategy="steps". Should be an integer or a float in range [0,1). 
    #If smaller than 1, will be interpreted as ratio of total training steps.
    overwrite_output_dir = True,
   # evaluation_strategy = "epoch", #"The evaluation strategy to adopt during training. "
  # save_strategy = "no",
    push_to_hub = False,
    output_dir = output_dir,
    report_to="tensorboard",
   # optim="paged_adamw_8bit",
)

In [15]:
from trl import SFTTrainer
trainer_sft =  SFTTrainer(
    model = model,
    train_dataset=dataset,
#    eval_dataset=dataset['valid'],
    peft_config = bullama.util.get_lora_config(),
    dataset_text_field = "texto",
    max_seq_length = 1024,
    tokenizer=tokenizer,
    args=training_args
)

Map:   0%|          | 0/1590 [00:00<?, ? examples/s]

In [16]:
trainer_sft.train()
trainer_sft.save_model(f"{model_dir}/{exp_name}")
tokenizer.save_pretrained(f"{model_dir}/{exp_name}")

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/navarro/miniconda3/envs/py39/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/navarro/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  w

Step,Training Loss
1,2.370800
2,1.943000
3,1.774300
4,2.185100
5,1.594300
6,1.400100
7,1.601500
8,1.785300
9,1.771700
10,1.698700


/home/navarro/miniconda3/envs/py39/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/navarro/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/navarro/miniconda3/envs/py39/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of 

('models/plaintext_teste/tokenizer_config.json',
 'models/plaintext_teste/special_tokens_map.json',
 'models/plaintext_teste/tokenizer.json')